<a href="https://colab.research.google.com/github/bf777/MesoNet/blob/master/mesonet_demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MesoNet
Welcome to MesoNet, a toolbox for segmenting mesoscale calcium images! This notebook will take you through all the steps needed to process your own calcium imaging dataset using our models.

In [ ]:
# Use tensorflow 1.x (supported by DeepLabCut)
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

In [ ]:
# Clone MesoNet repository
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone -s https://$GITHUB_AUTH@github.com/bf777/MesoNet.git

## Prepare inputs

In [ ]:
!mkdir mesonet_inputs/

Now, we will copy sample data (11 images) from the git repository to the inputs folder. If, instead, you would like to upload your own images create a folder inside `mesonet_inputs` and put your images inside that folder; then, skip the next cell. 

In [ ]:
# If you have your own input data that you've put into a subfolder under
# mesonet_inputs, don't run this cell.
%cp -r MesoNet/tests/test_input/ mesonet_inputs/

In [ ]:
!pip install deeplabcut

Because of how DeepLabCut operates, you now need to restart your runtime (under the Runtime menu).

In [ ]:
%cd MesoNet/

In [ ]:
# Utility to install mesonet package and associated requirements (will be replaced with pip later)
!pip install matplotlib==3.1.3
!python setup.py install

# pip install mesonet

We will now pull information from the OSF repository containing the DeepLabCut and U-Net models for our code.

In [ ]:
!pip install osfclient

In [ ]:
#@title OSF Username
osf_username = 'brandon.forys@alumni.ubc.ca' #@param {type: "string"}


In [ ]:
# Get OSF info
from getpass import getpass
import os
osf_password = getpass('OSF password')
os.environ['OSF_USER'] = osf_username
os.environ['OSF_PASSWORD'] = osf_password

In [ ]:
!osf -p svztu -u ${OSF_USER} fetch 6_Landmark_estimation_model/atlas-DongshengXiao-2020-08-03.zip mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip

In [ ]:
!unzip mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip

In [ ]:
!osf -p svztu -u ${OSF_USER} fetch 7_U-Net_model/DongshengXiao_bundary.hdf5 MesoNet/mesonet/models/DongshengXiao_bundary.hdf5

Now, input the information about your input and output images, as well as the U-Net and DeepLabCut models that you would like to use. The default values will use the test data that we've included in the MesoNet git repository (in `MesoNet/mesonet/tests/test_input`). If you're using your own input data, replace `input_filename` below with the name of the folder in `mesonet_inputs` containing your input data.

In [ ]:
#@title Input information for the model
input_filename = 'test_input'  #@param {type: "string"}
output_filename = 'mesonet_outputs'  #@param {type: "string"}
model_name = 'DongshengXiao_bundary.hdf5' #@param {type: "string"}
dlc_model_name = 'atlas-DongshengXiao-2020-08-03'  #@param {type: "string"}

In [ ]:
!mkdir /content/mesonet_outputs

In [ ]:
%cd /content/

In [ ]:
import os
# Set up filepaths based on your inputs
input_file = os.path.join('/content','mesonet_inputs', input_filename)
output_file = os.path.join('/content', output_filename)
model = os.path.join('MesoNet', 'mesonet', 'models', model_name)
dlc_config = os.path.join(dlc_model_name, 'config.yaml')

Now that we've told Colab where to find the input and output folders, let's define the configuration file!

In [ ]:
# We need to make sure that DeepLabCut doesn't run with a GUI (which isn't
# supported in Colab).
os.environ["DLClight"]="True"

# Import mesonet and define the configuration file according to your chosen
# inputs
import mesonet
config_file = mesonet.config_project(input_file, output_file, 'test',
                                     mat_save=False, config=dlc_config, 
                                     model=model)

The config file (by default in `mesonet_outputs`) contains information about how MesoNet will run. We'll be using this config file as an input to the last two functions needed to run MesoNet.

In [ ]:
config_file

Next, we'll use the U-net model to identify the outer edges of the cortex:

In [ ]:
mesonet.predict_regions(config_file)

In [ ]:
# Set this environment variable to help MesoNet find the git repo location
os.environ["MESONET_GIT"]='/content/MesoNet/mesonet/'

Lastly, we will identify and use cortical landmarks to align the atlas to the brain:

In [ ]:
mesonet.predict_dlc(config_file)

Congratulations, you're all done! You can now check the outputs in the `mesonet_outputs` folder. The segmented brain data can be found in `mesonet_outputs/output_mask`!